---
title: Classification
author: Pikall Nikolas
jupyter: 
  kernelspec:
    name: "01_classification"
    language: "python"
    display_name: "01_classification"
eval: false
---

In [ ]:
from datetime import datetime, timedelta

import xarray as xr
import pystac_client
import stackstac
import odc.stac
from odc.geo.geobox import GeoBox
from dask.diagnostics import ProgressBar
from rasterio.enums import Resampling

import cmcrameri as cmc
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import pandas as pd

# Python Script
from functions import preprocess_data_to_classify

# Scikit Learn
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import matplotlib.colors as colors

In this Notebook, we will use ``Scikit learn`` to classify an ``Sentinel-2`` Image. We will use two different classifiers and compare the results. 

## Data Acquisition
Before we start, we need to load the data. We will use ``ODC Stac`` to get data from Earth Search by Element 84. Here we define the area of interest and the time frame, aswell as the EPSG code and the resolution.


In [ ]:
dx = 0.0006  # 60m resolution
epsg = 4326

# Spatial extent
latmin, latmax = 47.86, 48.407
lonmin, lonmax = 16.32, 16.9
bounds = (lonmin, latmin, lonmax, latmax)
minx, miny, maxx, maxy = bounds
geom = {
    'type': 'Polygon',
    'coordinates': [[
       [minx, miny],
       [minx, maxy],
       [maxx, maxy],
       [maxx, miny],
       [minx, miny]
    ]]
}

# Temporal extent
year = 2024
month = 5
day = 1
delta = 10

start_date = datetime(year, month, day)
end_date = start_date + timedelta(days=delta)
date_query = start_date.strftime("%Y-%m-%d") + "/" + end_date.strftime("%Y-%m-%d")

# Search for Sentinel-2 data
items = pystac_client.Client.open(
    "https://earth-search.aws.element84.com/v1"
).search(
    intersects=geom,
    collections=["sentinel-2-l2a"],
    datetime=date_query,
    limit=100,
).item_collection()

print(len(items), "scenes found")

We have now found a couple of scenes that we can use for our analysis. Now we will load the data directly into an ``xarray`` dataset.

In [ ]:
# define a geobox for my region
geobox = GeoBox.from_bbox(bounds, crs=f"epsg:{epsg}", resolution=dx)

In [ ]:
# lazily combine items
ds_odc = odc.stac.load(
    items,
    bands=["scl", "red", "green", "blue", "nir"],
    chunks={'time': 5, 'x': 600, 'y': 600},
    geobox=geobox,
    resampling="bilinear",
)

In [ ]:
# actually load it
with ProgressBar():
    ds_odc.load()

Next we define a mask to filter out the clouds and the areas with no data and can plot a composite median image of the data.

In [ ]:
# define a mask for valid pixels (non-cloud)

def is_valid_pixel(data):
    # include only vegetated, not_vegitated, water, and snow
    return ((data > 3) & (data < 7)) | (data==11)

ds_odc['valid'] = is_valid_pixel(ds_odc.scl)
ds_odc.valid.sum("time").plot()

In [ ]:
def avg(ds):
    return (ds / ds.max() * 2)

# compute the masked median
rgb_median = (
    ds_odc[['red', 'green', 'blue']]
    .where(ds_odc.valid)
    .to_dataarray(dim="band")
    .transpose(..., "band")
    .median(dim="time")
    .astype(int)
)
rgb_comp = avg(rgb_median)
plot = rgb_comp.plot.imshow(rgb="band", figsize=(8, 8))
plot.axes.set_title(f"RGB - Median Composite\n{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}")
plt.show()

To get an first impression of the data, we can calculate the NDVI (Normalized Difference Vegetation Index) and plot it. This gives us a good overview of the vegetation in the area.

In [ ]:
# Normalized Difference Vegetation Index (NDVI)
def normalized_difference(a, b):
    return (a - b*1.) / (a + b)

ndvi = normalized_difference(ds_odc.nir, ds_odc.red)
ndvi.median(dim="time").plot.imshow(cmap='cmc.cork').axes.set_title('NDVI')
plt.show()

## Classification 

### Data Preparation
In order to start with the actual classification, we need to prepare some data. Since this is a supervised classification, we need to have some training data. We have two ``Geojson`` Files which contain areas that are either forest or not forest. We will use these to train our classifiers. Additionally from the loaded Dataset we will extract the bands that we want to use for the classification.


In [ ]:
forest_path = Path("shapefiles/forest.geojson")
nonforest_path = Path("shapefiles/nonforest.geojson")

In [ ]:
ds_class = (
    ds_odc[['red', 'green', 'blue', 'nir']]
    .where(ds_odc.valid)
    .median(dim="time")
)
ds_cf = avg(ds_class)

single_img = ds_cf.fillna(0)
X_train, X_test, y_train, y_test = preprocess_data_to_classify(ds=single_img, feature_path=forest_path, nonfeature_path=nonforest_path)

In [ ]:
# Naive Bayes
nb = GaussianNB()
nb_test = nb.fit(X_train, y_train)
nb_predict = nb.predict(X_test)

In [ ]:
print("NAIVE BAYES: \n "+ classification_report(y_test, nb_predict))

In [ ]:
print("NAIVE BAYES: \n ",confusion_matrix(y_test, nb_predict), "\n")

In [ ]:
# Creating 
bands = []
for band in ["red", "green", "blue", "nir"]:
    bands.append(single_img[band].values)
    
image_data = np.stack(bands, axis=2)

In [ ]:
# Reshape the image data
num_of_pixels = single_img.sizes['longitude'] * single_img.sizes['latitude']
num_of_bands = len(bands)
X_image_data = image_data.reshape(num_of_pixels, num_of_bands)

# Predict and reshape the image
nb_predict_img = nb.predict(X_image_data)
nb_predict_img = nb_predict_img.reshape(single_img.sizes['latitude'], single_img.sizes['longitude'])

In [ ]:
single_img['NB-forest'] = xr.DataArray(nb_predict_img, dims=['latitude', 'longitude'], coords={'longitude': single_img['longitude'], 'latitude': single_img['latitude']})

In [ ]:
alpha = 1	
cmap_green = colors.ListedColormap([(1, 1, 1, alpha), 'green'])

plot = single_img['NB-forest'].plot.imshow(cmap=cmap_green, cbar_kwargs={'ticks': [0.25,0.75]})
cbar = plot.colorbar
cbar.set_ticklabels(['non-forest', 'forest'])
plot.axes.set_title('Naive Bayes Classification')
plt.show()

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf_test = rf.fit(X_train, y_train)
rf_predict = rf.predict(X_test)

rf_predict_img = rf.predict(X_image_data)
rf_predict_img = rf_predict_img.reshape(single_img.sizes['latitude'], single_img.sizes['longitude'])

single_img['RF-forest'] = xr.DataArray(rf_predict_img, dims=['latitude', 'longitude'], coords={'longitude': single_img['longitude'], 'latitude': single_img['latitude']})

plot = single_img['RF-forest'].plot.imshow(cmap=cmap_green, cbar_kwargs={'ticks': [0.25,0.75]})
cbar = plot.colorbar
cbar.set_ticklabels(['non-forest', 'forest'])
plot.axes.set_title('Random Forest Classification')
plt.show()

In [ ]:
print("RANDOM FOREST: \n "+ classification_report(y_test, rf_predict))

In [ ]:
print("RANDOM FOREST: \n ",confusion_matrix(y_test, rf_predict), "\n")

In [ ]:
# cmap_trio = colors.ListedColormap(['gainsboro' ,'limegreen', 'mediumseagreen', 'darkgreen'])
cmap_trio = colors.ListedColormap(['whitesmoke' ,'indianred', 'goldenrod', 'darkgreen'])


double_clf = (single_img['NB-forest'] + 2*single_img['RF-forest'])

fig, ax = plt.subplots()
cax = ax.imshow(double_clf, cmap=cmap_trio, interpolation='none')

# Add a colorbar with custom tick labels
cbar = fig.colorbar(cax, ticks=[1*0.375, 3*0.375, 5*0.375, 7*0.375])
cbar.ax.set_yticklabels(['None', 'Naive Bayes', 'Random Forest', 'Both'])
ax.set_title('Classification Comparisson')
ax.set_axis_off()
plt.show()

In [ ]:
# Plot only one class, either None (0), Naive Bayes (1), Random Forest (2), or Both (3)
fig, axs = plt.subplots(2,2, figsize=(8,8))
ax = axs.ravel()

for i in range(4):
    ax[i].imshow(double_clf==i, cmap='cmc.oleron_r', interpolation='none')
    category = ['by None', 'only by Naive Bayes', 'only by Random Forest', 'by Both'][i]
    title = 'Areas classified ' + category
    ax[i].set_title(title)
    ax[i].set_axis_off()

plt.tight_layout()

In [ ]:
counts = {}
for num in range(1,4): # Change the start of the range to 0 if you want to include the 'None' class
    num_2_class = {0: 'None', 1: 'Naive Bayes', 2: 'Random Forest', 3: 'Both'}
    counts[num_2_class[num]] = int((double_clf == num).sum().values)

class_counts_df = pd.DataFrame(list(counts.items()), columns=['Class', 'Count'])

In [ ]:
class_counts_df['Percentage'] = (class_counts_df['Count'] / class_counts_df['Count'].sum())*100
ax = class_counts_df.plot.bar(x='Class', y='Percentage', rot=0, color='darkgreen', ylim=(0,100), title='Classified Areas per Classificator (%)')

# Annotate the bars with the percentage values
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')